In [1]:
from vec_math import *
nltk.download('punkt')

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package punkt to /home/micha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# English tests

In [2]:
# model = fasttext.train_unsupervised('../data/fil9')
# model = fasttext.train_unsupervised('data/fil9', minn=2, maxn=5, dim=300)
# model.save_model("models/fil9.bin")

model = fasttext.load_model("models/fil9.bin")

x = "metallica"
y = "abba"
vec1 = model.get_word_vector(x)
vec2 = model.get_word_vector(y)

print(vec1.shape)
print(model.get_nearest_neighbors(x), end="\n\n")

# print(model.get_analogies("berlin", "germany", "czechia"))

print("scipy cosine:", cosine_sim_scipy(vec1, vec2))
print("numpy cosine:", cosine_sim_numpy(vec1, vec2))

(100,)
[(0.8522698879241943, 'megadeth'), (0.7971707582473755, 'pantera'), (0.7917349934577942, 'deftones'), (0.7859595417976379, 'frontman'), (0.7821397185325623, 'mustaine'), (0.7773374915122986, 'sepultura'), (0.7772756218910217, 'hetfield'), (0.7724277377128601, 'radiohead'), (0.7667999863624573, 'ozzy'), (0.7642661333084106, 'band')]

scipy cosine: 0.40885651111602783
numpy cosine: 0.40885654


# Czech tests

In [ ]:
model = fasttext.load_model('models/cc.cs.300.bin')

In [4]:
print(model.get_word_vector("univerzita").shape)

(300,)


## get_sentence_vector() vs average of word embeddings

In [5]:
sent1 = "Main purpose of this thesis is to show the functionality."
sent2_similar = "The primary objective of a publication was to demonstrate the performance."
sent3_different = "Another problem with this configuration was to Cook some delicious dinner!"
sent4_absurd = "Amidst the lush jungle, a majestic tiger stealthily prowled through the undergrowth."

In [6]:
# with fasttext.get_sentence_vector()
similarity12 = cosine_sim_scipy(model.get_sentence_vector(sent1), model.get_sentence_vector(sent2_similar))
similarity13 = cosine_sim_scipy(model.get_sentence_vector(sent1), model.get_sentence_vector(sent3_different))
similarity14 = cosine_sim_scipy(model.get_sentence_vector(sent1), model.get_sentence_vector(sent4_absurd))

print("Similar:", round(similarity12, 3))
print("Different:", round(similarity13, 3))
print("Absurd:", round(similarity14, 3))

Similar: 0.867
Different: 0.796
Absurd: 0.695


In [7]:
# just take the average of word embeddings
similarity12 = cosine_sim_scipy(get_average_vector(model, sent1), get_average_vector(model, sent2_similar))
similarity13 = cosine_sim_scipy(get_average_vector(model, sent1), get_average_vector(model, sent3_different))
similarity14 = cosine_sim_scipy(get_average_vector(model, sent1), get_average_vector(model, sent4_absurd))

print("Similar:", round(similarity12, 3))
print("Different:", round(similarity13, 3))
print("Absurd:", round(similarity14, 3))

Similar: 0.922
Different: 0.798
Absurd: 0.769


## Testovácí otázky

In [8]:
df = pd.read_excel("../data/Q50_questions.xlsx")
print(df)

     Unnamed: 0                                           question  class
0             0                          Co je duševní vlastnictví      0
1             1                     Co známená duševní vlastnictví      0
2             2                             Duševní vlastnictví je      0
3             3                     Definice duševního vlastnictví      0
4             4                  Jak definovat duševní vlastnictví      0
..          ...                                                ...    ...
557         557      Doba řízení národní přihlášky ochranné známky     48
558         558  V jakém časovém horizontu po registraci ochran...     48
559         559  Za jak dlouho bude národní ochranná známka zar...     48
560         560     Uplynutí doby platnosti zápisu ochranné známky     49
561         561  Upozorňuje Úřad na uplynutí doby platnosti záp...     49

[562 rows x 3 columns]


In [9]:
sent = "Kdo provádí výběr výroků" # class 33
print(classify_one_sentence(df, model, sent))

(33, 'Kdo provádí oceňování vynálezů')


### Leave-one-out cross-validation
Classify one testing sample on other training data and compute empirical error.

In [10]:
x_train, x_test = pd_df_to_train_test_sets(df, test_percentage=1)

In [11]:
# Classify by nearest neighbour (does not consider the same question as the classified one)
print("Test error:", get_classification_error(df, model, x_test))

Test error: 0.104982206405694


In [12]:
class2question_dict = get_class2questions_dict(df)
print(class2question_dict[38])

['Obnova', 'Prodloužení platnosti', 'Udržovací poplatky', 'Jak požádat o obnovu nebo prodloužení platnosti u ochranných známek', 'Jak požádat o obnovu nebo prodloužení platnosti u užitných vzorů', 'Jak požádat o obnovu nebo prodloužení platnosti u průmyslových vzorů', 'Jak postupovat při placení udržovacích poplatků u patentů']


In [13]:
# Average all questions in one class from training data and classify based on this average
# m.get_sentence_vector works better

sent = "Kdo provádí výběr výroků" # class 33
# sent = "Opis listiny" # 40
# sent = "Je možné název webových stránek ochránit jako ochrannou známku" # 47
# sent = "Kde hledat pomoc v případě porušení práv duševního vlastnictví" # 5
print(classify_one_sentence_based_on_average(model, class2question_dict, sent))

33


In [14]:
print("Test error:", get_classification_error_with_averages(class2question_dict, model, x_test))

Test error: 0.14946619217081847
